In [2]:
import pymysql
# 引入python中的traceback模块，跟踪错误
import traceback
import sys

db = MysqlUtil() 

In [15]:
db = pymysql.connect(host='192.168.8.161', user='root', password='Wzh010310*', db='notebook')

In [3]:


class MysqlUtil():
    def __init__(self):
        '''
            初始化方法，连接数据库
        '''
        host = '192.168.8.236'  # 主机名
        user = 'root'  # 数据库用户名
        password = '8008'  # 数据库密码
        database = 'weibo'  # 数据库名称
        self.db = pymysql.connect(host=host, user=user, password=password, db=database)  # 建立连接
        self.cursor = self.db.cursor(cursor=pymysql.cursors.DictCursor)  # 设置游标，并将游标设置为字典类型

    def insert(self, sql):
        '''
            插入数据库
            sql:插入数据库的sql语句
        '''
        try:
            # 执行sql语句
            self.cursor.execute(sql)
            # 提交到数据库执行
            self.db.commit()
        except Exception:  # 方法一：捕获所有异常
            # 如果发生异常，则回滚
            print("发生异常", Exception)
            self.db.rollback()
        finally:
            # 最终关闭数据库连接
            self.db.close()

    def fetchone(self, sql):
        '''
            查询数据库：单个结果集
            fetchone(): 该方法获取下一个查询结果集。结果集是一个对象
        '''
        try:
            # 执行sql语句
            self.cursor.execute(sql)
            result = self.cursor.fetchone()
        except:  # 方法二：采用traceback模块查看异常
            # 输出异常信息
            traceback.print_exc()
            # 如果发生异常，则回滚
            self.db.rollback()
            result = 'None'

        finally:
            # 最终关闭数据库连接
            self.db.close()
        return result

    def fetchall(self, sql):
        '''
            查询数据库：多个结果集
            fetchall(): 接收全部的返回结果行.
        '''
        try:
            # 执行sql语句
            self.cursor.execute(sql)
            results = self.cursor.fetchall()
        except:  # 方法三：采用sys模块回溯最后的异常
            # 输出异常信息
            info = sys.exc_info()
            print(info[0], ":", info[1])
            # 如果发生异常，则回滚
            self.db.rollback()
            results = 'None'
        finally:
            # 最终关闭数据库连接
            self.db.close()
        return results

    def delete(self, sql):
        '''
            删除结果集
        '''
        try:
            # 执行sql语句
            self.cursor.execute(sql)
            self.db.commit()
        except:  # 把这些异常保存到一个日志文件中，来分析这些异常
            # 将错误日志输入到目录文件中
            f = open("\log.txt", 'a')
            traceback.print_exc(file=f)
            f.flush()
            f.close()
            # 如果发生异常，则回滚
            self.db.rollback()
        finally:
            # 最终关闭数据库连接
            self.db.close()

    def update(self, sql):
        '''
            更新结果集
        '''
        try:
            # 执行sql语句
            self.cursor.execute(sql)
            self.db.commit()
        except:
            # 如果发生异常，则回滚
            self.db.rollback()
        finally:
            # 最终关闭数据库连接
            self.db.close()


In [14]:
import time
time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

'2023-01-11 13:55:26'

In [4]:
sql_1=f'select status_province,COUNT(status_province) as t from weiboevents WHERE title="杭州两所中小学出现新冠阳性学生" GROUP by status_province'
db = MysqlUtil()
articles = db.fetchall(sql_1)

In [5]:
articles

[{'status_province': '', 't': 14},
 {'status_province': '佛山', 't': 5},
 {'status_province': '信阳', 't': 5},
 {'status_province': '兰州', 't': 5},
 {'status_province': '北京', 't': 73},
 {'status_province': '南京', 't': 12},
 {'status_province': '合肥', 't': 5},
 {'status_province': '唐山', 't': 5},
 {'status_province': '大连', 't': 5},
 {'status_province': '天水', 't': 10},
 {'status_province': '安阳', 't': 5},
 {'status_province': '广州', 't': 5},
 {'status_province': '德阳', 't': 2},
 {'status_province': '成都', 't': 10},
 {'status_province': '无锡', 't': 3},
 {'status_province': '昆明', 't': 5},
 {'status_province': '晋中', 't': 8},
 {'status_province': '杭州', 't': 42},
 {'status_province': '武汉', 't': 5},
 {'status_province': '淄博', 't': 5},
 {'status_province': '湖州', 't': 3},
 {'status_province': '潍坊', 't': 10},
 {'status_province': '珠海', 't': 5},
 {'status_province': '福州', 't': 8},
 {'status_province': '苏州', 't': 5},
 {'status_province': '赣州', 't': 10},
 {'status_province': '遵义', 't': 5},
 {'status_province': '

In [6]:
def clearTxt(line:str):
    if(line != ''):
        line = line.strip()
        # 去除文本中的英文和数字
        line = re.sub("[a-zA-Z0-9]", "", line)
        # 去除文本中的中文符号和英文符号
        line = re.sub("[\s+\.\!\/_,$%^*(+\"\'；：“”．]+|[+——！，。？?、~@#￥%……&*（）]+", "", line)
        return line
    return None

#文本切割
def sent2word(line):
    segList = jieba.cut(line,cut_all=False)
    segSentence = ''
    for word in segList:
        if word != '\t':
            segSentence += word + " "
    return segSentence.strip()